In [ ]:
%matplotlib notebook
#%matplotlib inline

import matplotlib.pyplot as plt
import rasterio
import numpy as np
import math
import importlib


import pyproj
import xarray as xr
import fiona
import shapely
import shapely.geometry
import shapely.ops

from numpy import pi, r_, c_, vstack, hstack

import kk.dggs
from kk.dggs import DGGS
from kk.tools import polygon_path, spread_num_samples, gen_pts_from_distribution
from kk.uitools import plot_bbox, add_cell_plot


In [ ]:
dg = DGGS()

In [ ]:
import importlib

importlib.reload(kk.dggs)
dg = kk.dggs.DGGS()

In [ ]:
import kk.uitools
importlib.reload(kk.uitools)

from kk.uitools import add_cell_plot, plot_bbox

In [ ]:
def get_affine(fname):
    with rasterio.open(fname) as f:
        return f.affine

def get_poly_lines():
    with fiona.open('/Users/kirill/wk/dggs/qgis/data/cstauscd_l.shp') as f:
        return [np.r_[ pydash.get(p, 'geometry.coordinates') ].T 
                for p in f if pydash.get(p, 'properties.FEAT_CODE') == 'tile_edge']

In [ ]:
filename = '/Users/kirill/wk/dggs/data/abs/apg16e1_1_1_0.tif'

In [ ]:
A = get_affine(filename)
ds = xr.open_rasterio(filename)
ds.attrs['affine'] = A
ds.values[ds.values<0] = np.nan

src = ds.isel(band=0)
ds.sum(), src.sum()

In [ ]:
plt.figure()
src.plot.imshow()

xx_, yy_ = polygon_path(ds.x, ds.y)
plt.plot(xx_, yy_, 'y.', markersize=0.1)
plt.axis('tight')
plt.axis('equal')

# S06003363  149 x 3996
# R36006000 5669 x 2862
# Q88208355  181 x  450

add_cell_plot('R36006000', 'r-', w=5669, h=2862, crs=ds.crs)
add_cell_plot('S06003363', 'g-', w=149 , h=3996, crs=ds.crs)
add_cell_plot('Q88208355', 'm-', w=181 , h=450 , crs=ds.crs)

In [ ]:
rr = {}

for roi in dg.compute_overlap(8, *polygon_path(ds.x, ds.y), ds.crs):
    print(f'Processing: {roi}')
    wrp = dg.mk_warper(roi, src_crs=ds.crs)
    im = wrp(ds.values[0], ds.affine, math.nan)
    if np.isnan(im).all():
        print('Skipping -- all nans')
    else:
        rr[str(roi.addr)] = im
    print('...done')
    

for addr, im in rr.items():
    plt.figure()
    plt.imshow(im)
    plt.title(addr + ': {}'.format(np.nansum(im)))

In [ ]:
def merge_extents(e1, e2):
    if e1 is None:
        return e2
    
    return [min(e1[0], e2[0]), max(e1[1], e2[1]),
            min(e1[2], e2[2]), max(e1[3], e2[3])]

def plot_all(south_square=0, north_square=0):
    dh = dg.mk_display_helper(south_square=south_square, north_square=north_square)
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    extents = None

    for addr, im in rr.items():
        im, ee = dh(addr, im) 
        extents = merge_extents(extents, ee)
        ax.imshow(im, extent=ee)

    ax.set_xlim(*extents[:2])
    ax.set_ylim(*extents[2:])
    return fig

fig = plot_all(south_square=3)


```
S06003363  149 x 3996
Q88208355  181 x  450
R36006000 5669 x 2862
```

In [ ]:
prj = pyproj.Proj(ds.crs)

lx,ly = prj(*polygon_path(ds.x, ds.y), inverse=True)

lx_range = np.r_[lx.min(), lx.max()]
ly_range = np.r_[ly.min(), ly.max()]


fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(lx,ly,'b.', markersize=0.1)
ax.plot(*polygon_path(lx_range, ly_range), 'r-')

cell_styles=dict(N='m-',S='k-', O='r-', P='g-', Q='b-', R='c-')

for c,extents in dg.top_level_extents.items():
    print(c, extents)
    plot_bbox(extents, cell_styles[c], ax=ax)

plt.axis('equal')
